## 데이터셋 로드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Kwargs/전처리/S-Oil.csv')
print(df.head())

                                         title                 date  \
0                 ET투자뉴스SOil 희망은 있다 매수유지신한금융투자  2019-01-28 10:29:28   
1                       특징주 정유주 국제유가 급등에 동반 상승  2019-02-18 09:18:31   
2                      태양광제조업수소경제..문재인의 산업정책지도  2019-01-17 14:33:55   
3  한경로보뉴스 SOil 수주공시 정유제품 공급계약 2.61조 매출액대비 12.5  2019-02-19 11:45:17   
4                      오스만 에쓰오일 대표 성추행 혐의로 검찰행  2019-01-31 21:40:01   

                                             content category  
0  신한금융투자에서 28일 SOil010950에 대해 희망은 있다라며 투자의견을 매수유...       경제  
1  국제유가 급등에 정유주들이 동반 상승하고 있다.18일 오전 9시12분 현재 SK이노...       경제  
2  the3002018.10월 군산 새만금 시작..경북경남울산 방문 마쳐문재인 대통령이...       정치  
3  02월 19일 SOil010950은 수주공시를 발표했다.SOil 수주공시 개요 정유...       경제  
4  지난달 용산 모처서 여성 신체접촉 혐의오스만 알 감디 에쓰오일SOil 대표가 성추행...       사회  


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8102 entries, 0 to 8101
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     8102 non-null   object
 1   date      8102 non-null   object
 2   content   8102 non-null   object
 3   category  8102 non-null   object
dtypes: object(4)
memory usage: 253.3+ KB


## BERTopic 라이브러리 설치
KoBERT로 모델 변경

In [ ]:
!pip install bertopic

In [5]:
from bertopic import BERTopic
from transformers import AutoModel, AutoTokenizer

# KoBERT 모델과 토크나이저 로드 (모델 이름은 예시입니다. 실제 모델명을 확인해야 합니다.)
model_name = "monologg/kobert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# BERTopic에 KoBERT 모델 적용
topic_model = BERTopic(embedding_model=(model, tokenizer))

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

## 토큰화 및 불용어 제거

In [14]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 35.5 MB/s eta 0:00:00


In [15]:
from konlpy.tag import Okt
import pandas as pd

# Okt 토크나이저 초기화
okt = Okt()

# 불용어 리스트 로드 함수
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = [line.strip() for line in file.readlines()]
    return stopwords

stopwords_path = '/content/drive/MyDrive/Kwargs/stopwords.txt'  # 불용어 리스트 파일 경로
stopwords = load_stopwords(stopwords_path)

# 텍스트에서 불용어 제거 함수 (Okt 버전)
def remove_stopwords_with_okt(text, stopwords):
    # Okt를 사용한 토큰화
    tokens = okt.morphs(text)
    # 불용어 제거
    filtered_tokens = [token for token in tokens if token not in stopwords]
    # 토큰들을 다시 문자열로 합치기
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# 데이터셋의 각 문서에 대해 불용어 제거 수행
filtered_doclist = [remove_stopwords_with_okt(doc, stopwords) for doc in df['content']]

# 결과 출력
for doc in filtered_doclist:
    print(doc)


신 한 금융투자 28일 SOil 010950 대해 희망 은 있다라며 투자 의견 수 유지 제시 하였고 목표 주가 로는 133000원 내놓았다 . 신 한 금융투자 이응 주 애널 리스트 동 종목 대하 제시 한 수 유지 의견 은 신 한 금융투자 직전 매매 의견 그대로 유지 하는 이고 전 년도 말 매매 의견 기준 볼때 도 역시 수 의견 계속 유지 되고 있는 상태 이다 . 최근 분기 내 발표 된 전체 증권사 리포트 컨센서스 비교 해볼 경우 오늘 발표 된 투자 의견 은 대 체적 평균 치 해당 하는 수준 의견 은 동 종목 대한 전체 적 인 흐름 크게 벗어나지 않은 분석 되고 . 목표 주가 추이 살펴보면 한동안 정체 구간 보낸 후 목표 가가 하향 조정 된 모습 이다 . 전일 종가 기준 볼때 동 종목 현 주가 는 제시 된 목표 대비 33.8 추가 상승 여력 있다는 해석 제시 되고 . Report briefing 신 한 금융투자 SOil 010950 대해 4분 기 영업 이익 은 1195억원 적자 전환 불가피하다 . 유가 급락 따른 대규모 재고 관련 손실 때문 이다 . 9월 기준 Dubai 평균 유가 는 배럴 당 77달러 달 했으나 12월 말 기준 유가 는 53달러 그쳤다 . 정유 업 황도 좋지 못 했다 . 4분 기 정제 마진 은 전 분기 대비 1.4달러 배럴 하락 했다라고 분석 했다 . 신 한 금융투자 11월 가동 된 신규 고도화 설비 도 실적 큰 도움 되지 못 했을 가능성 크다 . 신규 설비 는 BC 라는 부가가치 제품 휘발유 석 유제품 프로필렌 유도체 석유화학 제품 변환 하는 설비 다라 고 밝혔다 . 한편 단 기적 정유 시 황 은 어렵다 . 1 하반기 갈수록 정제 마진 은 개선 2 H 19 IMO 2020 효과 기대 될 가능성 크다 . 2 신규 설비 가동 도 이익 증가 한몫 하겠다 . 32019년 기준 예상 배당 수익률 은 6.1 다라 고 전망 했다 . Report statistics 신 한 금융투자 동 종목 대한 최근 1년 동안의 투자 의견 은 전체 적 큰 변화 없이 유지 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



경기 침체 공포 코스피 1년 8 개월 만에 2300 선 아래 마감 했다 . 달러 강세 원 달러 환율 은 이틀 째 1300원 위 거래 마쳤다 . 6일 코스피 는 전 거래 일보 다 49.77 포인트 2.13 내린 2292.01 장 마쳤다 . 코스피 2300 아래 마감 한 은 2020년 10월 30일 2267.15 이후 처음 이다 . 기관 6230억원 외국인 3138억원 순 매도 며 지수 끌어내렸다 . 개인 은 8965억원 사들였다 . 코스닥 은 전 거래 일보 다 6.32 포인트 0.84 떨어진 744.63 거래 마쳤다 . 시 총 상위 권 삼성 1.40 SK 하이닉스 0.43 성 바이오 직스 0.62 현대차 2.82 삼성 SDI 2.61 LG 화학 1.17 하락 마감 했다 . 국제 유가 급락 SK 이노베이션 5.26 SOil 9.31 정 유주 낙폭 더 컸다 . LG 에너지 솔루션 은 일본 상용차 업체 1조원 규모 배터리 공급 한다는 소식 2.49 올랐다 . 날 유가 증권 시장 오른 종목 수 는 177 개 내린 종목 수 는 697 개였다 . 김석환 미래에셋 증권 연구원 은 유럽 경기 침체 우려 부각 되고 원 달러 환율 장 중 1310원 돌파 하는 강 달러 기조 심화 되면서 외국인 도세 확대 되고 대형주 위주 하락 했다고 밝혔다 . 원 달러 환율 은 이틀 째 1300원 위 마감 했다 . 날 서울 외환시장 원 달러 환율 은 6.0원 오른 1306.3원 거래 마쳤다 . 날 원 달러 환율 은 개장 직후 1311.00 오르며 지난달 30일 기록 했던 장 중 연고 점 1303.7원 도 경신 했다 . 연 고 점 은 2009년 7월 13일 고가 기준 1315.0원 이후 약 13년 만에 가장 높은 수준 이다 . 이후 원 달러 환율 은 외환 당국 개입 대한 경계 감 소 폭 내리며 1306원 선에서 횡보했다 . 글로벌 경기 침체 우려 확산 안전 자산 선호 심리 강해진 국내 시장 에도 영향 줬다 . 경기 둔화 원자재 수요 도 줄어들 이란 우려 국제 유가 급락 한 불안 심리 키웠다 . 5일

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## BERTopic

In [22]:
# BERTopic 모델 훈련
doclist = df['content'].tolist()
topics, probs = topic_model.fit_transform(doclist) # 토큰화 결과 : doc
# 주제 출력
topic_info = topic_model.get_topic_info()
print(topic_info)

# 특정 주제에 대한 대표 단어 출력
topic_number = 0  # 예시로 0번 주제를 선택
topic = topic_model.get_topic(topic_number)
print(f"Topic {topic_number} Representative Words and their Scores:")
for word, score in topic:
    print(f"{word}: {score}")

TypeError: Make sure that the documents variable is an iterable containing strings only.

In [18]:
import matplotlib.pyplot as plt
import pandas as pd
df = topic_info
# 주제 번호와 주제 이름을 결합하여 새로운 'Topic_Name' 열을 생성
df['Topic_Name'] = df['Topic'].astype(str) + ": " + df['Name'].apply(lambda x: x.split('_')[1])

In [19]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs,Topic_Name
0,-1,3579,-1_있다_것으로_대비_지난해,"[있다, 것으로, 대비, 지난해, 국내, 올해, 있는, 지난, 실적, 따르면]",[에쓰오일SOil010950이 올해 3분기 정유 제품 수요 호조에 따른 국제유가 상...,-1: 있다
1,0,516,0_있다_있는_것으로_국내,"[있다, 있는, 것으로, 국내, 대한, 지난, 올해, 지난해, 있습니다, 위해]",[의 대화에서 이렇게 말했다. 우리 부처가 최전선에 있지 않습니다. 3월21일 공개...,0: 있다
2,1,320,1_에쓰오일_에쓰오일은_밝혔다_말했다,"[에쓰오일, 에쓰오일은, 밝혔다, 말했다, 관계자는, 에쓰오일의, 서울, 통해, 위...",[에쓰오일SOil010950이 올해 3분기 유가 상승과 여름철 성수기 수요 호조에 ...,1: 에쓰오일
3,2,203,2_soil은_soil_soil이_통해,"[soil은, soil, soil이, 통해, 전달했다, 위한, 위해, 밝혔다, 함께...",[파이낸셜뉴스 SOIL은 한국사회복지협의회와 서울 중구 프레지던트호텔에서 2019 ...,2: soil은
4,3,168,3_오른_배럴당_거래되고_오전,"[오른, 배럴당, 거래되고, 오전, 정유주가, 국제유가가, 뉴욕상업거래소에서, 거래...",[산유국들의 추가 감산이 대규모로 이뤄질 것으로 예상되면서 국제유가 급등으로 인해 ...,3: 오른
...,...,...,...,...,...,...
103,102,12,102_석유제품_1분기_수출국_수출,"[석유제품, 1분기, 수출국, 수출, 3분기, 감소한, 배럴로, 수출량은, 수출한,...",[정유업계가 올해 1분기 석유제품 수출 호황을 맞았다. 수출 물량이 전년 동기보다 ...,102: 석유제품
104,103,11,103_요소수_수소_위한_친환경,"[요소수, 수소, 위한, 친환경, 사업목적에, 수소연료전지, 추가한다, 요소수를, ...",[차BEV의 변속기 및 감속기에 최적화된 윤활유 4종을 개발완료했다. 이후 국내외 ...,103: 요소수
105,104,11,104_soil010950은_공시했다_공급계약을_매출액의,"[soil010950은, 공시했다, 공급계약을, 매출액의, 체결했다고, 납사, 노희...",[SOIL010950은 대한유화주식회사와 8500억원 규모의 납사 공급계약을 체결했...,104: soil010950은
106,105,10,105_감사인_감사인을_soil의_주기적,"[감사인, 감사인을, soil의, 주기적, soil은, 삼일pwc로, 재지정을, 재...",[SOil010950이 이달 4거래일 연속 신저가를 기록한 가운데 개인투자자들은 출...,105: 감사인


In [20]:
topic_info.to_csv('/content/drive/MyDrive/Kwargs/뉴스 크롤링/csv/S-Oil_news_topic_bertopic.csv', index=False)

In [ ]:
# 1. 불용어리스트를 찾아서 제거 한 후에
# 2. 스포츠 뉴스가 정상 제거 될 수 있는지 판단할 수 있는 기업으로 테스트 다시해보고
# 3. 관련이 없는 주제를 구분 (구분하는 기준은?)
# 4. 구분하여 제거 적절성 검토 후 필요한 경우 제거
